In [11]:
import numpy as np
import os
import yaml
import random

In [12]:
input_file = './inputs/input_5ag_0dy_3sta_seed1.yaml'
dy_file = './dy_obstacles/dy_5ag_0dy_3sta_seed1.yaml'

In [13]:
N = 5
# static_obs_ratio = 0.6
num_static_obs = 3
map_size_x = 17
map_size_y = 8
x_min = 0
y_min = 0
seed = 1


num_dynamic_obs = 1
step_num_from = 1
step_num_to = 2
manhadon_dist_min = 1

In [14]:
random.seed(seed)

input_f = open(input_file, 'a')

input_dict = {'agents': []}

start_ls = []
end_ls = []

input_f.write('agents:\n')
print('agents:')
for agent_num in range(N):
    while True:
        start_x = random.choice(range(x_min, map_size_x))
        start_y = random.choice(range(y_min, map_size_y))
        end_x = random.choice(range(x_min, map_size_x))
        end_y = random.choice(range(y_min, map_size_y))
        if [start_x, start_y] not in start_ls and [end_x, end_y] not in end_ls:
            start_ls.append([start_x, start_y])
            end_ls.append([end_x, end_y])
            break
        else:
            continue
    input_dict['agents'].append({'start': [start_x, start_y], 'goal': [end_x, end_y], 'name': "agent" + str(agent_num)})
    input_f.write("-   start: " + str([start_x, start_y]) + '\n')
    input_f.write("    goal: " + str([end_x, end_y]) + '\n')
    input_f.write("    name: agent" + str(agent_num) + '\n')
    print("-   start: ", [start_x, start_y])
    print("    goal: ", [end_x, end_y])
    print("    name: agent" + str(agent_num))


# print(start_ls)
# print(end_ls)

agents:
-   start:  [4, 1]
    goal:  [8, 1]
    name: agent0
-   start:  [15, 7]
    goal:  [15, 6]
    name: agent1
-   start:  [6, 1]
    goal:  [15, 0]
    name: agent2
-   start:  [12, 6]
    goal:  [0, 7]
    name: agent3
-   start:  [8, 3]
    goal:  [3, 5]
    name: agent4


In [15]:
num_static_obs = num_static_obs
obs_ls = []
input_f.write('map:\n')
input_f.write('    dimensions: [' + str(map_size_x) + ', ' + str(map_size_y) + ']\n')
input_f.write('    obstacles:' + '\n')
print('map:')
print('    dimensions: [', map_size_x, ', ', map_size_y, ']')
print('    obstacles:')
for i in range(num_static_obs):
    while True:
        obs_x = random.choice(range(map_size_x))
        obs_y = random.choice(range(map_size_y))
        if [obs_x, obs_y] not in start_ls and [obs_x, obs_y] not in end_ls and [obs_x, obs_y] not in obs_ls:
            obs_ls.append([obs_x, obs_y])
            break
        else:
            continue
    input_f.write('    - !!python/tuple ' + str([obs_x, obs_y]) + '\n')
    print('    - !!python/tuple ', [obs_x, obs_y])


# print(obs_ls)

map:
    dimensions: [ 17 ,  8 ]
    obstacles:
    - !!python/tuple  [0, 0]
    - !!python/tuple  [12, 3]
    - !!python/tuple  [13, 0]


In [16]:



input_f.write('obstacles:\n')
for i in range(num_dynamic_obs):
    input_f.write('-   start: [0, 0]\n')
    input_f.write('    goal: [32, 32]\n')
    input_f.write('    name: obstacle' + str(i) + '\n')
        
input_f.close()

dy_f = open(dy_file, 'a')

random.seed(seed)

dy_f.write('schedule:\n')
print('schedule:')
for dyn_obs in range(num_dynamic_obs):
    dy_f.write('  obstacle' + str(dyn_obs) + ':\n')
    print('  obstacle' + str(dyn_obs) + ':')
    while True:
        step_x = random.choice(range(map_size_x))
        step_y = random.choice(range(map_size_y))
        step = [step_x, step_y]
        
        is_bigger = True
        for start_pt in start_ls:
            manhadon_dist = abs(start_pt[0] - step_x) + abs(start_pt[1] - step_y)
            if manhadon_dist < manhadon_dist_min:
                is_bigger = False
                break
                
        if is_bigger:
            if step not in start_ls and step not in end_ls and step not in obs_ls:
                dy_f.write('  - t: ' + str(0) + '\n')
                dy_f.write('    x: ' + str(step_x) + '\n')
                dy_f.write('    y: ' + str(step_y) + '\n')
                print('  - t: ', 0)
                print('    x: ', step_x)
                print('    y: ', step_y)
                break
    step_num = random.choice(range(step_num_from, step_num_to))
    for step_ in range(step_num):
        while True:
            direction = random.choice(range(5)) # 0 => up | 1 => down | 2 => left | 3 => right | 4 => still 
            if direction == 0:
                step_x_ = step_x
                step_y_ = step_y + 1 
            elif direction == 1:
                step_x_ = step_x
                step_y_ = step_y - 1 
            elif direction == 2:
                step_y_ = step_y
                step_x_ = step_x - 1 
            elif direction == 3:
                step_y_ = step_y
                step_x_ = step_x + 1 
            else:
                step_y_ = step_y
                step_x_ = step_x

            step = [step_x_, step_y_]

            if step not in end_ls and step not in obs_ls and (0 <= step_x_ <= 31) and (0 <= step_y_ <= 31):
                dy_f.write('  - t: ' + str((step_ + 1)) + '\n')
                dy_f.write('    x: ' + str(step_x) + '\n')
                dy_f.write('    y: ' + str(step_y) + '\n')
                print('  - t: ', (step_ + 1))
                print('    x: ', step_x_)
                print('    y: ', step_y_)
                step_x = step_x_
                step_y = step_y_
                break
dy_f.close()

schedule:
  obstacle0:
  - t:  0
    x:  16
    y:  3
  - t:  1
    x:  17
    y:  3
